In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pypfopt import expected_returns, risk_models, BlackLittermanModel
import yfinance as yf
import pypfopt
import os

In [2]:
data = {}
start_date = '2020-01-01'
end_date = '2020-12-31'

In [3]:
data['VTI'] = yf.download('VTI', period='max')
data['VTV'] = yf.download('VTV', period='max')
data['VOE'] = yf.download('VOE', period='max')
data['VBR'] = yf.download('VBR', period='max')
data['GBIL'] = yf.download('GBIL', period='max')
data['JPST'] = yf.download('JPST', period='max')
data['VNQ'] = yf.download('VNQ', period='max')
data['VPU'] = yf.download('VPU', period='max')
#data['BITO'] = yf.download('BITO', period='max')


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [4]:
for ticker in data:
    data[ticker].to_csv(ticker + '.csv')

for ticker in data:
    data[ticker]['percentageChange'] = data[ticker]['Close'].pct_change()*10
    data[ticker]['percentageChange'] = data[ticker]['percentageChange'].fillna(0)

In [5]:
onlyClosePrices = pd.DataFrame()
for ticker in data:
    onlyClosePrices[ticker] = data[ticker]['Close']

In [6]:
S = pypfopt.risk_models.CovarianceShrinkage(onlyClosePrices).ledoit_wolf()

In [7]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from pandas import read_csv
import math
from keras.models import Sequential, load_model
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from keras.layers.core import Dense, Activation, Dropout
from sklearn.model_selection import train_test_split
import tensorflow as tf
import keras
import time #helper libraries

In [8]:
scalar = MinMaxScaler(feature_range=(-1, 1))

In [9]:
data['VTI'] = scalar.fit_transform(data['VTI'].dropna())
data['VTV'] = scalar.fit_transform(data['VTV'].dropna())
data['VOE'] = scalar.fit_transform(data['VOE'].dropna())
data['VBR'] = scalar.fit_transform(data['VBR'].dropna())
data['GBIL'] = scalar.fit_transform(data['GBIL'].dropna())
data['JPST'] = scalar.fit_transform(data['JPST'].dropna())
data['VNQ'] = scalar.fit_transform(data['VNQ'].dropna())
data['VPU'] = scalar.fit_transform(data['VPU'].dropna())
#data['BITO'] = scalar.fit_transform(data['BITO'].dropna())

In [10]:
def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back), -2]
        dataX.append(a)
        dataY.append(dataset[i + look_back, 0])
    return np.array(dataX), np.array(dataY)

In [11]:
trainingData = {}
testData = {}

In [12]:
for ticker in data:
    trainingData[ticker], testData[ticker] = train_test_split(data[ticker], test_size=0.05, shuffle=False)

In [13]:
trainX, trainY = {}, {}
testX, testY = {}, {}
timeStep = 100
for ticker in data:
    trainX[ticker], trainY[ticker] = create_dataset(trainingData[ticker], timeStep)
    testX[ticker], testY[ticker] = create_dataset(testData[ticker], timeStep)

In [14]:
trainX['VTI'].shape

(4920, 100)

In [15]:
model = Sequential()
model.add(LSTM(units=100, return_sequences=True, input_shape=(trainX['VTI'].shape[1], 1)))
model.add(Dropout(0.2))
model.add(LSTM(units=50, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=50, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=50))
model.add(Dropout(0.2))
model.add(Dense(units=1))
model.compile(loss='mean_squared_error', optimizer=tf.keras.optimizers.SGD(learning_rate=0.1, momentum=0.9), metrics=[tf.keras.metrics.RootMeanSquaredError()])
#model.compile(loss='mean_squared_error', optimizer=tf.keras.optimizers.SGD(learning_rate=0.1, momentum=0.9), metrics=[tf.keras.metrics.RootMeanSquaredError()])

In [17]:
VERSION = '1'
for ticker in data:
    model.fit(trainX[ticker], trainY[ticker], epochs=50, batch_size=3, verbose=1, validation_data=(testX[ticker], testY[ticker]))
    model.save('models/'+VERSION + '/' + ticker + '.h5')
    model.save_weights(ticker + '_weights.h5')

Epoch 1/50
1640/1640 [==============================] - 70s 42ms/step - loss: 0.0029 - root_mean_squared_error: 0.0537 - val_loss: 0.9780 - val_root_mean_squared_error: 0.9889
Epoch 2/50
1640/1640 [==============================] - 72s 44ms/step - loss: 0.0026 - root_mean_squared_error: 0.0507 - val_loss: 0.9801 - val_root_mean_squared_error: 0.9900
Epoch 3/50
1640/1640 [==============================] - 72s 44ms/step - loss: 0.0023 - root_mean_squared_error: 0.0479 - val_loss: 1.0726 - val_root_mean_squared_error: 1.0357
Epoch 4/50
1640/1640 [==============================] - 73s 44ms/step - loss: 0.0024 - root_mean_squared_error: 0.0491 - val_loss: 0.9662 - val_root_mean_squared_error: 0.9830
Epoch 5/50
1640/1640 [==============================] - 73s 44ms/step - loss: 0.0027 - root_mean_squared_error: 0.0522 - val_loss: 0.9393 - val_root_mean_squared_error: 0.9692
Epoch 6/50
1640/1640 [==============================] - 73s 44ms/step - loss: 0.0023 - root_mean_squared_error: 0.0484 -

In [ ]:
model = load_model('VTI.h5')

In [22]:
viewDict = {}

In [24]:
models = {}
for ticker in data:
    print(ticker)
    models[ticker] = load_model(ticker + '.h5')
    viewDict[ticker] = model.predict(testX[ticker])

VTI
VTV
VOE
VBR
GBIL


ValueError: Unexpected result of `predict_function` (Empty batch_outputs). Please use `Model.compile(..., run_eagerly=True)`, or `tf.config.run_functions_eagerly(True)` for more information of where went wrong, or file a issue/bug to `tf.keras`.

In [ ]:
trainX['VTI'].shape

In [ ]:
viewDict['VTI']

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pypfopt
import yfinance as yf
import pandas_datareader as pdr
import seaborn as sns
import random

In [25]:
netAssets = {}
for ticker in data:
    netAssets[ticker] = yf.Ticker(ticker).info['totalAssets']

KeyboardInterrupt: 

In [ ]:
netAssets

In [ ]:
csvAssets = pd.read_csv('assets.csv', index_col=0)

In [ ]:
csvAssets['0']['VTI']

In [ ]:
delta = pypfopt.black_litterman.market_implied_risk_aversion(pd.Series(netAssets), risk_free_rate=0.05796)

In [ ]:
marketPrior = pypfopt.black_litterman.market_implied_prior_returns(netAssets, delta, S)

In [ ]:
viewDict = {}
confidence = []
for ticker in data:
    pred = models[ticker].predict(testX[ticker])
    viewDict[ticker] = (pred[-1,0] - testX[ticker][-2, 0])/testX[ticker][-2, 0]
    confidence.append(1-model.evaluate(testX[ticker], testY[ticker])[1])

In [ ]:
viewDict

In [ ]:
bl = pypfopt.black_litterman.BlackLittermanModel(S, pi=marketPrior, absolute_views=viewDict, rho=delta)

In [ ]:
bl_confi = pypfopt.BlackLittermanModel(S, absolute_views=viewDict, omega="idzorek", rho=delta, view_confidences=confidence)

In [ ]:
bl_return_confi = bl_confi.bl_returns()
bl_return_confi.name = 'BL Returns with Confidence'

In [ ]:
S_bl_confi = bl_confi.bl_cov()

In [ ]:
ef = pypfopt.EfficientFrontier(bl_return_confi, S_bl_confi, weight_bounds=(0, 0.2))

In [ ]:
#ef.add_objective('Sharpe', 'max')
weights = ef.min_volatility()
ef.portfolio_performance(verbose=True)

In [ ]:
import json
json.dumps(weights)

In [ ]:
wt_min_vola = pd.DataFrame([weights], columns=weights.keys()).T*100
print ('Weights in Percentage ********************')
print(wt_min_vola.round(4))

In [ ]:
data